In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
month_eod_df = pd.read_csv('data_options/spx_eod_2022/spx_eod_202210.csv')
#month_eod_df.reset_index()
month_eod_df.where((month_eod_df['[STRIKE_DISTANCE]'] < (month_eod_df['[STRIKE]']*0.35)), inplace=True)
month_eod_df.dropna(inplace=True)
month_eod_df[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']]

In [ ]:
month_eod_df[520:540][['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']]

In [ ]:
month_eod_df.shape

In [ ]:
month_eod_df.columns

In [ ]:
cnt = 0
for index, row in month_eod_df.iterrows():
    if cnt > 5:
        break

    print(row)
    cnt += 1

In [ ]:
calls_df = month_eod_df.filter(items=['[QUOTE_UNIXTIME]', '[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE]', '[STRIKE_DISTANCE_PCT]', '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]', '[C_BID]', '[C_ASK]'])
puts_df = month_eod_df.filter(items=['[QUOTE_UNIXTIME]', '[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE]', '[STRIKE_DISTANCE_PCT]', '[EXPIRE_UNIX]', '[DTE]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]', '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_LAST]', '[P_SIZE]', '[P_BID]', '[P_ASK]'])

In [ ]:
calls_df[10:20]

In [ ]:
calls_df.shape

In [ ]:
calls_df['[C_VOLUME]'].notna().value_counts()[True]

In [ ]:
pd.isna(calls_df['[C_VOLUME]'])

In [ ]:
## Replace Empty rows with NaN values.

calls_df['[C_IV]'].replace({' ':np.nan},inplace=True)
calls_df['[C_VOLUME]'].replace({' ':np.nan},inplace=True)
calls_df['[C_LAST]'].replace({' ':np.nan},inplace=True)
calls_df['[C_SIZE]'].replace({' ':np.nan},inplace=True)
calls_df['[C_BID]'].replace({' ':np.nan},inplace=True)
calls_df['[C_ASK]'].replace({' ':np.nan},inplace=True)

In [ ]:
## Replace Empty rows with NaN values.

puts_df['[P_IV]'].replace({' ':np.nan},inplace=True)
puts_df['[P_VOLUME]'].replace({' ':np.nan},inplace=True)
puts_df['[P_LAST]'].replace({' ':np.nan},inplace=True)
puts_df['[P_SIZE]'].replace({' ':np.nan},inplace=True)
puts_df['[P_BID]'].replace({' ':np.nan},inplace=True)
puts_df['[P_ASK]'].replace({' ':np.nan},inplace=True)

In [ ]:
62740 + 632

In [ ]:
# Rename Columns
rename_dict = {'[QUOTE_UNIXTIME]':'QUOTE_UNIXTIME', '[UNDERLYING_LAST]':'UNDERLYING_LAST', '[STRIKE]':'STRIKE', '[STRIKE_DISTANCE]':'STRIKE_DISTANCE', '[STRIKE_DISTANCE_PCT]':'STRIKE_DISTANCE_PCT', '[EXPIRE_UNIX]':'EXPIRE_UNIX', '[DTE]':'DTE', '[C_DELTA]':'DELTA', '[C_GAMMA]':'GAMMA', '[C_VEGA]':'VEGA', '[C_THETA]':'THETA', '[C_RHO]':'RHO', '[C_IV]':'IV', '[C_VOLUME]':'VOLUME', '[C_LAST]':'LAST', '[C_SIZE]':'SIZE', '[C_BID]':'BID', '[C_ASK]':'ASK',}
rename_dict2 = {'[QUOTE_UNIXTIME]':'QUOTE_UNIXTIME', '[UNDERLYING_LAST]':'UNDERLYING_LAST', '[STRIKE]':'STRIKE', '[STRIKE_DISTANCE]':'STRIKE_DISTANCE', '[STRIKE_DISTANCE_PCT]':'STRIKE_DISTANCE_PCT', '[EXPIRE_UNIX]':'EXPIRE_UNIX', '[DTE]':'DTE', '[P_DELTA]':'DELTA', '[P_GAMMA]':'GAMMA', '[P_VEGA]':'VEGA', '[P_THETA]':'THETA', '[P_RHO]':'RHO', '[P_IV]':'IV', '[P_VOLUME]':'VOLUME', '[P_LAST]':'LAST', '[P_SIZE]':'SIZE', '[P_BID]':'BID', '[P_ASK]':'ASK',}
calls_df = calls_df.rename(columns=rename_dict)
puts_df = puts_df.rename(columns=rename_dict2)
print(puts_df[:5])

##### Remove Expired worthless contracts

In [ ]:
# Remove Expired Worhless Calls
calls_df = calls_df[(calls_df['DTE'] != 0) | (calls_df['STRIKE'] < calls_df['UNDERLYING_LAST'])]
# See Rmoving contracts 
#calls_df[(calls_df['DTE'] == 0) & (calls_df['STRIKE'] > calls_df['UNDERLYING_LAST'])]

# Remove Expired Worhless Puts
puts_df = puts_df[(puts_df['DTE'] != 0) | (puts_df['STRIKE'] > puts_df['UNDERLYING_LAST'])]
# See Rmoving contracts 
#puts_df[(puts_df['DTE'] == 0) & (puts_df['STRIKE'] < puts_df['UNDERLYING_LAST'])]

In [ ]:
calls_df.dtypes

In [ ]:
# IV BID etc. Attribute is has NaNs.
print(calls_df['IV'].notna().value_counts()[True])
print(puts_df['BID'].notna().value_counts()[True])

# Size Attribute is String. > No NaNs
print(calls_df['SIZE'].notna().value_counts()[True])
print(puts_df['SIZE'].notna().value_counts()[True])

In [ ]:
# Filling NaN values with 0
calls_df = calls_df.fillna(0)
puts_df = puts_df.fillna(0)

In [ ]:
calls_df[10:20]

In [ ]:
# Add Type for Calls and Puts for classification purposes.
# Calls: 1 & Puts: 0
calls_df['TYPE'] = 'CALL'
puts_df['TYPE'] = 'PUT'

In [ ]:
puts_df.shape

#### Alternative method. Just Calls and no Puts with no Price(Calculate using BSM) and y=IV
##### -- Calculate Price from resulting IV.

In [ ]:
#calls_df = calls_df.sort_values(by=['QUOTE_UNIXTIME', 'DTE', 'STRIKE']).reset_index()
calls_df[190:195]
#calls_df.dtypes


In [ ]:
calls_df.dtypes

#### Transformations to CALLS Dataframe

In [ ]:
# Splitting SIZE attribute and converting to int.
calls_df[['SIZE1', 'SIZE2']] = calls_df['SIZE'].str.split(' x ', expand=True)
calls_df.SIZE1 = calls_df.SIZE1.astype(np.int64)
calls_df.SIZE2 = calls_df.SIZE2.astype(np.int64)

# Convert Strike dtype to int. Otherwise float output is received for strike which is invalid.
#calls_df.STRIKE = calls_df.STRIKE.astype(np.int64)

# Drop redundant colunmns. DTE can serve the same purpose 
calls_df = calls_df.drop('QUOTE_UNIXTIME', axis=1)
calls_df = calls_df.drop('EXPIRE_UNIX', axis=1)

# Drop Strike Distance because Strike is there.
# Convert Strike to integer.
calls_df = calls_df.drop('STRIKE_DISTANCE', axis=1)
calls_df['STRIKE'] = calls_df['STRIKE'].astype(np.int64)

# Drop Size because Size has been split into 2 columns above.
calls_df = calls_df.drop('SIZE', axis=1)

# Only calls used so TYPE is redundant.
#calls_df = calls_df.drop('TYPE', axis=1)

# Converting String formatting of ASK, BID, LAST, IV, VOLUME
calls_df['ASK'] = calls_df['ASK'].astype(np.float64)
calls_df['BID'] = calls_df['BID'].astype(np.float64)
calls_df['LAST'] = calls_df['LAST'].astype(np.float64)
calls_df['IV'] = calls_df['IV'].astype(np.float64)

# Converting String formatting VOLUME to float and the int.
calls_df['VOLUME'] = calls_df['VOLUME'].astype(np.float64)
calls_df['VOLUME'] = calls_df['VOLUME'].astype(np.int64)

# Days to Expiry is an integer value
calls_df['DTE'] = calls_df['DTE'].astype(np.int64)


In [ ]:

# Moving Type column to the end
cols = ['UNDERLYING_LAST', 'STRIKE', 'STRIKE_DISTANCE_PCT',
       'DTE', 'DELTA', 'GAMMA', 'VEGA', 'THETA', 'RHO', 'VOLUME',
       'SIZE1', 'SIZE2', 'BID', 'ASK', 'IV', 'LAST']
calls_df = calls_df[cols]

In [ ]:
calls_df[100:102]

##### Convert to X and y for training. Save IV column as iv for later calculation and evaluation.

In [ ]:
X_num = calls_df.iloc[:,:-1]
y = calls_df.iloc[:,-1:]

In [ ]:
X_num[100:102]

In [ ]:
X_num_train, X_num_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=1)
X_num_train, X_num_val, y_train, y_val = train_test_split(X_num_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
X_num_train[:2]
#iv_train[:2]

In [ ]:
iv_train = X_num_train.iloc[:, -1:]
iv_test = X_num_test.iloc[:, -1:]
iv_val = X_num_val.iloc[:, -1:]

X_num_train = X_num_train.iloc[:, :-1]
X_num_test = X_num_test.iloc[:, :-1]
X_num_val = X_num_val.iloc[:, :-1]

In [ ]:
type(X_num_train)

In [ ]:
X_num_train.shape

In [ ]:
# Coverting Dataframes to .npy for better storage.
X_num_train = X_num_train.to_numpy()
X_num_test = X_num_test.to_numpy()
X_num_val = X_num_val.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [ ]:
# Save the Numpy arrays for training.

with open('tab-ddpm/tab-ddpm-main/data/options/X_num_train.npy', 'wb') as f:
    np.save(f, X_num_train)
with open('tab-ddpm/tab-ddpm-main/data/options/X_num_test.npy', 'wb') as f:
    np.save(f, X_num_test)
with open('tab-ddpm/tab-ddpm-main/data/options/X_num_val.npy', 'wb') as f:
    np.save(f, X_num_val)

with open('tab-ddpm/tab-ddpm-main/data/options/y_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('tab-ddpm/tab-ddpm-main/data/options/y_test.npy', 'wb') as f:
    np.save(f, y_test)
with open('tab-ddpm/tab-ddpm-main/data/options/y_val.npy', 'wb') as f:
    np.save(f, y_val)

##### Check Synthetic Data for only Calls [/Puts]

In [ ]:
synthe_X_num = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/X_num_train.npy')
synthe_y = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/y_train.npy')

In [ ]:
synthe_X_num.shape

In [ ]:
cols = ['[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE_PCT]', '[DTE]', '[C_DELTA]',
       '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_VOLUME]', '[C_SIZE1]', '[C_SIZE2]',
       '[C_BID]', '[C_ASK]']
len(cols)

In [ ]:
synthe_X_df = pd.DataFrame(synthe_X_num, columns = cols)

In [ ]:
synthe_X_df.shape

In [ ]:
synthe_X_df[20:25]

In [ ]:
synthe_X_df.sort_values(by=['[DTE]', '[STRIKE]'])[3155:3163]

##### Calls and Puts dataframe together.

In [ ]:
# Join Calls & Puts dataframes into a sinlgle dataframe
c_split_p_eod = pd.concat([calls_df, puts_df]).sort_values(by=['QUOTE_UNIXTIME', 'DTE', 'STRIKE']).reset_index()

In [ ]:
c_split_p_eod[190:210]

In [ ]:
# Splitting SIZE attribute and converting to int.
c_split_p_eod[['SIZE1', 'SIZE2']] = c_split_p_eod['SIZE'].str.split(' x ', expand=True)
c_split_p_eod.SIZE1 = c_split_p_eod.SIZE1.astype(np.int64)
c_split_p_eod.SIZE2 = c_split_p_eod.SIZE2.astype(np.int64)


In [ ]:
# Checking value in SIZE1 is int
c_split_p_eod['SIZE1'][209]

In [ ]:
# Convert Strike dtype to int. Otherwise float output is received for strike which is invalid.
c_split_p_eod.STRIKE = c_split_p_eod.STRIKE.astype(np.int64)
c_split_p_eod.dtypes

In [ ]:
# Drop redundant colunmns. DTE can serve the same purpose 
c_split_p_eod = c_split_p_eod.drop('QUOTE_UNIXTIME', axis=1)
c_split_p_eod = c_split_p_eod.drop('EXPIRE_UNIX', axis=1)

# Drop Strike because Strike Distance is there.
c_split_p_eod = c_split_p_eod.drop('STRIKE_DISTANCE', axis=1)

# Drop Size because Size has been split into 2 columns above.
c_split_p_eod = c_split_p_eod.drop('SIZE', axis=1)

In [ ]:
c_split_p_eod.dtypes
#c_split_p_eod.shape

In [ ]:
# Converting String formatting of ASK, BID, LAST, IV, VOLUME
c_split_p_eod['ASK'] = c_split_p_eod['ASK'].astype(np.float64)
c_split_p_eod['BID'] = c_split_p_eod['BID'].astype(np.float64)
c_split_p_eod['LAST'] = c_split_p_eod['LAST'].astype(np.float64)
c_split_p_eod['IV'] = c_split_p_eod['IV'].astype(np.float64)

# Converting String formatting VOLUME to float and the int.
c_split_p_eod['VOLUME'] = c_split_p_eod['VOLUME'].astype(np.float64)
c_split_p_eod['VOLUME'] = c_split_p_eod['VOLUME'].astype(np.int64)

# Days to Expiry is an integer value
c_split_p_eod['DTE'] = c_split_p_eod['DTE'].astype(np.int64)
#_ = c_split_p_eod['ASK'].map(type) == float
#_.value_counts()

##### Tuning attempt.
###### Strike change to int for consistency of what strikes are.

In [ ]:
#c_split_p_eod['VOLUME'] = c_split_p_eod['VOLUME'].astype(int)
#_ = c_split_p_eod['DTE'].map(type) == int
#_.value_counts()

In [ ]:
c_split_p_eod.shape
#c_split_p_eod[:5]

In [ ]:
# Moving Type column to the end
cols = ['UNDERLYING_LAST', 'STRIKE', 'STRIKE_DISTANCE_PCT',
       'DTE', 'DELTA', 'GAMMA', 'VEGA', 'THETA', 'RHO', 'VOLUME',
       'SIZE1', 'SIZE2', 'BID', 'ASK', 'TYPE', 'LAST', 'IV']
c_split_p_eod = c_split_p_eod[cols]

#### Splitting Dataframe into X and y for training purposes

In [ ]:
X_num = c_split_p_eod.iloc[:,:-1]
y = c_split_p_eod.iloc[:,-1:]

In [ ]:
X_num.shape

In [ ]:
y[:5]
y.shape

In [ ]:
X_num_train, X_num_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=1)
X_num_train, X_num_val, y_train, y_val = train_test_split(X_num_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
X_num_test[:5]

In [ ]:
# Get Categorical Columns for Multinomial Loss.
X_cat_train = X_num_train.loc[:, 'TYPE']
X_cat_test = X_num_test.loc[:, 'TYPE']
X_cat_val = X_num_val.loc[:, 'TYPE']

# Drop TYPE from Numerical table.
X_num_train = X_num_train.drop('TYPE', axis=1)
X_num_test = X_num_test.drop('TYPE', axis=1)
X_num_val = X_num_val.drop('TYPE', axis=1)

In [ ]:
X_num_val.shape

In [ ]:
# Coverting Dataframes to .npy for better storage.
X_num_train = X_num_train.to_numpy()
X_num_test = X_num_test.to_numpy()
X_num_val = X_num_val.to_numpy()

X_cat_train = X_cat_train.to_numpy()
X_cat_test = X_cat_test.to_numpy()
X_cat_val = X_cat_val.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [ ]:
X_cat_train[:5]

In [ ]:
# Reshape Categorical Arrays. Single Column or 1D Array not accepted by model
X_cat_train = X_cat_train.reshape(-1,1)
X_cat_test = X_cat_test.reshape(-1,1)
X_cat_val = X_cat_val.reshape(-1,1)

#### Saving X and y for train, test and val.

In [ ]:
with open('tab-ddpm/tab-ddpm-main/data/options/X_num_train.npy', 'wb') as f:
    np.save(f, X_num_train)
with open('tab-ddpm/tab-ddpm-main/data/options/X_num_test.npy', 'wb') as f:
    np.save(f, X_num_test)
with open('tab-ddpm/tab-ddpm-main/data/options/X_num_val.npy', 'wb') as f:
    np.save(f, X_num_val)

with open('tab-ddpm/tab-ddpm-main/data/options/X_cat_train.npy', 'wb') as f:
    np.save(f, X_cat_train)
with open('tab-ddpm/tab-ddpm-main/data/options/X_cat_test.npy', 'wb') as f:
    np.save(f, X_cat_test)
with open('tab-ddpm/tab-ddpm-main/data/options/X_cat_val.npy', 'wb') as f:
    np.save(f, X_cat_val)

with open('tab-ddpm/tab-ddpm-main/data/options/y_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('tab-ddpm/tab-ddpm-main/data/options/y_test.npy', 'wb') as f:
    np.save(f, y_test)
with open('tab-ddpm/tab-ddpm-main/data/options/y_val.npy', 'wb') as f:
    np.save(f, y_val)

In [ ]:
#c_split_p_eod['VOLUME'].notna().value_counts()[True]

##### Checking Generated data

In [ ]:
synthe_X_num = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/X_num_train.npy')

In [ ]:
# Pick_allow disabled causing error. 
# Workaround to get X_cat

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
synthe_X_cat = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/X_cat_train.npy')

# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
synthe_y = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/y_train.npy')

In [ ]:
synthe_X_num.shape
#synthe_y.shape

In [ ]:
synthe_X = np.append(synthe_X_num, synthe_X_cat, axis=1)

In [ ]:
synthe_X[:2].shape

In [ ]:
synthe_X_df = pd.DataFrame(synthe_X_num,
    columns = ['[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE_PCT]', '[DTE]', '[C_DELTA]',
       '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_VOLUME]', '[C_SIZE1]', '[C_SIZE2]',
       '[C_BID]', '[C_ASK]'])

In [ ]:
synthe_X_df[-10:-5]

In [ ]:
synthe_y[:5]

#### Calculating and Evaluating Greeks

In [ ]:
month_eod_df.columns

In [ ]:
month_eod_df[441:442]

In [ ]:
month_eod_df[[' [EXPIRE_DATE]', ' [STRIKE]', ' [UNDERLYING_LAST]', ' [DTE]']][month_eod_df[' [EXPIRE_DATE]'] == ' 2022-09-07']

In [ ]:
sp7_from_aug = month_eod_df.loc[(month_eod_df[' [EXPIRE_DATE]'] == ' 2022-09-07') & (month_eod_df[' [STRIKE]'] == 400)]

In [ ]:
sp7_from_sp

(month_eod_df[' [DTE]'] == 6.0)

In [ ]:
sp7_from_aug

In [ ]:
month_eod_df[[' [EXPIRE_DATE]', ' [DTE]']][23:24]

In [ ]:
month_eod_df[' [UNDERLYING_LAST]'][23]

In [ ]:
path_to_file = 'data_options/spy_eod_2022/spy_eod_20220'
sp7_400 = pd.DataFrame()

for i in range(1,10):
    file_path = path_to_file + str(i) + '.csv'
    print(file_path)
    month_eod_df = pd.read_csv(file_path)
    temp = month_eod_df.loc[(month_eod_df[' [EXPIRE_DATE]'] == ' 2022-09-07') & (month_eod_df[' [STRIKE]'] == 400)]
    sp7_400 = pd.concat([sp7_400, temp], ignore_index = True)

#month_eod_df = pd.read_csv('data_options/spy_eod_2022/spy_eod_202208.csv')

In [ ]:
# DATA PREP for C_LAST, C_BID, C_ASK
sp7_400[' [C_LAST]'].replace({' ':np.nan},inplace=True)
sp7_400[' [C_BID]'].replace({' ':np.nan},inplace=True)
sp7_400[' [C_ASK]'].replace({' ':np.nan},inplace=True)

sp7_400 = sp7_400.fillna(0)

sp7_400[' [C_LAST]'] = sp7_400[' [C_LAST]'].astype(np.float64)
sp7_400[' [C_BID]'] = sp7_400[' [C_BID]'].astype(np.float64)
sp7_400[' [C_ASK]'] = sp7_400[' [C_ASK]'].astype(np.float64)

In [ ]:
sp7_400[[' [QUOTE_DATE]', ' [UNDERLYING_LAST]', ' [DTE]', ' [C_DELTA]', ' [C_LAST]', ' [C_BID]', ' [C_ASK]', ' [STRIKE_DISTANCE]', ' [STRIKE]']]

In [ ]:
# Filling C_LAST == 0 into C_BID C_ASK average
sp7_400[' [C_LAST]'] = sp7_400[' [C_LAST]'].where(sp7_400[' [C_LAST]'] != 0, (sp7_400[' [C_BID]']+sp7_400[' [C_ASK]'])/2, axis=0)

In [ ]:
sp7_400[' [C_LAST]']

In [ ]:
sp7_400.columns

In [ ]:
sp7_400[[' [QUOTE_DATE]', ' [UNDERLYING_LAST]', ' [DTE]', ' [C_DELTA]', ' [C_LAST]', ' [C_BID]', ' [C_ASK]', ' [STRIKE_DISTANCE]', ' [STRIKE]']]

In [ ]:
sp7_400.shape

In [ ]:
sp7_400.iloc[2,]

##### Transform C_LAST

In [ ]:
#calc_delta = [0]
cum_underlast_diff = [0]
cum_clast_diff = [0]
last_underlast = 0
last_clast = 0


for i, row in sp7_400.iterrows():
    if i == 0:
        last_underlast = row[' [UNDERLYING_LAST]']
        last_clast = row[' [C_LAST]']
        #print('Yoooooooooooooo', last_clast, row[' [C_LAST]'])
        continue
    
    res1 = abs(row[' [UNDERLYING_LAST]'] - last_underlast) + cum_underlast_diff[-1]
    last_underlast = row[' [UNDERLYING_LAST]']
    cum_underlast_diff.append(res1)

    #print('here', last_clast)
    res2 = abs(row[' [C_LAST]'] - last_clast) + cum_clast_diff[-1]
    #print('Here', res2, last_clast, row[' [C_LAST]'], cum_clast_diff[-1])
    #print(res2)
    last_clast = row[' [C_LAST]']
    cum_clast_diff.append(res2)


#print(cum_underlast_diff[-1])
    

In [ ]:
#cum_underlast_diff
#sp7_400.shape
#sp7_400[' CUM_CLAST_DIFF'] = cum_clast_diff
#sp7_400[' CUM_UNDERLAST_DIFF'] = cum_underlast_diff

sp7_400[[' [QUOTE_DATE]', ' [UNDERLYING_LAST]', ' [DTE]', ' [C_DELTA]', ' [C_LAST]', ' [C_BID]', ' [C_ASK]', ' [STRIKE_DISTANCE]', ' [STRIKE]', ' CUM_UNDERLAST_DIFF', ' CUM_CLAST_DIFF', ' [C_IV]']]


#### Converting .txt to .csv

path_to_file = 'data_options/spx_eod_2022/spx_eod_202212.txt'
df_txt = pd.read_table(path_to_file, sep=', ')
df_txt.to_csv('data_options/spx_eod_2022/spx_eod_202212.csv')

In [ ]:
path_to_file_prefix = 'data_options/spx_eod_2022/spx_eod_2022'
for i in range(1,10):
    if i < 10:
        path_to_file = path_to_file_prefix + '0' + str(i) + '.txt'
    else:
        path_to_file = path_to_file_prefix + str(i) + '.txt'
    df_txt = pd.read_table(path_to_file, sep=', ')
    df_txt.to_csv(path_to_file_prefix + '0' + str(i) + '.csv')



#### GET SYNTHETIC DATA

In [ ]:
synthe_X_num = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/X_num_train.npy')


## UNCOMMENT IF DATATSET HAS CAT VALUES
'''
# Pick_allow disabled causing error. 
# Workaround to get X_cat

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
synthe_X_cat = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/X_cat_train.npy')

# restore np.load for future normal usage
np.load = np_load_old

synthe_X = np.append(synthe_X_num, synthe_X_cat, axis=1)
'''

# If Cat value in dataset, change synthe_X_num to synthe_X
synthe_X_df = pd.DataFrame(synthe_X_num,
    columns = ['[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE_PCT]', '[DTE]', '[C_DELTA]',
       '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_VOLUME]', '[C_SIZE1]', '[C_SIZE2]',
       '[C_BID]', '[C_ASK]'])

synthe_y = np.load('tab-ddpm/tab-ddpm-main/exp/options/ddpm_cb_best/y_train.npy')

#### Check BSM model

In [ ]:
check_df = pd.read_csv('data_options/spx_eod_2022/spx_eod_202201.csv')
check_df1 = pd.read_csv('data_options/spx_eod_2022/spx_eod_202202.csv')

In [ ]:
check_df.dropna(inplace=True)
check_df1.dropna(inplace=True)

In [ ]:
check_df = pd.concat([check_df, check_df1], ignore_index=True)

In [ ]:
check_df[79778:79782]

In [ ]:
check_df[['[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]']].notna().value_counts()
#pd.isna(check_df['[C_VOLUME]'])
#
#check_df.columns

In [ ]:
check_df[['[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]','[C_BID]', '[C_ASK]']].notna().value_counts()

In [ ]:
check_df = check_df.filter(items=['[QUOTE_DATE]', '[QUOTE_UNIXTIME]', '[EXPIRE_DATE]', '[EXPIRE_UNIX]', '[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE]', '[STRIKE_DISTANCE_PCT]', '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]', '[C_BID]', '[C_ASK]'])
check_df.dtypes

In [ ]:
check_df[-10020:-10010]

In [ ]:
check_df.shape

In [ ]:
check_df['[DTE]'] = check_df['[DTE]'].round()
check_df['[DTE]'] = check_df['[DTE]'].astype(np.int64)
check_df['[STRIKE]'] = check_df['[STRIKE]'].astype(np.int64)
check_df['[C_VOLUME]'] = check_df['[C_VOLUME]'].astype(np.int64)
check_df.dtypes

In [ ]:
check_df[['[C_ASK]']].notna().value_counts()

In [ ]:
check_df.where((check_df['[STRIKE_DISTANCE]'] < (check_df['[UNDERLYING_LAST]']*0.2)), inplace=True)
check_df.dropna(inplace=True)

In [ ]:
check_df.dtypes

In [ ]:
check_df[['[QUOTE_DATE]', '[QUOTE_UNIXTIME]', '[EXPIRE_DATE]', '[EXPIRE_UNIX]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]', '[C_VOLUME]']][6150:6155]

In [ ]:
check_df.columns

In [ ]:
#check_df.loc[:,['[STRIKE]', '[STRIKE_DISTANCE]', '[EXPIRE_UNIX]', '[DTE]', '[C_VOLUME]']]
#check_df.where((type(check_df['[C_VOLUME]']) == np.float64))
check_df.loc[3200:3300, ['[STRIKE]', '[STRIKE_DISTANCE]', '[EXPIRE_UNIX]', '[DTE]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]', '[C_BID]', '[C_ASK]']]

In [ ]:
check_df.shape

In [ ]:
check_df['[DTE]'].replace({' ':np.nan},inplace=True)
#check_df = check_df.fillna(0)
#check_df['[DTE]'] = check_df['[DTE]'].astype(np.int64)

In [ ]:
check_df[['[C_ASK]']].notna().value_counts()

calc_daily_var = check_df.loc[(check_df['[EXPIRE_DATE]'] == '2023-01-03') & (check_df['[STRIKE]'] == 3900)]
calc_daily_var.where(calc_daily_var['[DTE]'] > 4, inplace=True)
#calc_daily_var = calc_daily_var.iloc[::-1]


In [ ]:
calls_df = check_df.filter(items=['[QUOTE_UNIXTIME]', '[UNDERLYING_LAST]', '[STRIKE]', '[STRIKE_DISTANCE]', '[STRIKE_DISTANCE_PCT]', '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]', '[C_BID]', '[C_ASK]'])

In [ ]:
calls_df.shape

In [ ]:
#calc_daily_var = calc_daily_var[:-1]
#calc_daily_var['[MID_PRICE]'] = (calc_daily_var['[C_BID]'] + calc_daily_var['[C_ASK]'])/2
#calc_daily_var.drop('[MID_PRICE]', axis=1)

In [ ]:
calc_daily_var

In [ ]:
cnt = 0
previous_dte = -1
week_flag = False
for i, row in calc_daily_var.iloc[::-1].iterrows():

    if week_flag == True:
        print('Here', i)
        calc_daily_var.drop(i, inplace=True)

    if row['[QUOTE_DATE]'] == '2022-12-29':
        previous_dte = row['[DTE]']
        continue

    if (row['[DTE]'] - previous_dte) == 1:
        cnt += 1
        previous_dte = row['[DTE]']
    elif (row['[DTE]'] - previous_dte) > 1:
        cnt = 0
        previous_dte = row['[DTE]']

    if cnt == 4:
        week_flag = True

In [ ]:
calc_daily_var = calc_daily_var[:-1]
calc_daily_var[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']]

In [ ]:
calc_daily_var['[MID_PRICE]'] = (calc_daily_var['[C_BID]'] + calc_daily_var['[C_ASK]'])/2
calc_daily_var[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]', '[MID_PRICE]']]

In [ ]:
calc_daily_var

In [ ]:
calc_daily_var['[MID_PRICE]'] = (calc_daily_var['[C_BID]'] + calc_daily_var['[C_ASK]'])/2

previous_mid = -1
percent_chge = []
for i, row in calc_daily_var.iterrows():
    if row['[QUOTE_DATE]'] == '2022-12-19':
        previous_mid = row['[MID_PRICE]']
        percent_chge.append(0)
        continue
    
    today_chnge = ((row['[MID_PRICE]'] - previous_mid)*100)/previous_mid
    previous_mid = row['[MID_PRICE]']
    percent_chge.append(today_chnge)

calc_daily_var['[PERCENT_CHANGE]'] = percent_chge
calc_daily_var.drop('[MID_PRICE]', axis=1)


In [ ]:
calc_daily_var[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]', '[PERCENT_CHANGE]']]

In [ ]:
(calc_daily_var['[PERCENT_CHANGE]'][1:].std()/100) * (252**(1/2))
#calc_daily_var['[PERCENT_CHANGE]'].std()

##### EXAMPLE from check_df

In [ ]:
check_df[30010:30020]
#check_df.shape

In [ ]:
#check_df.columns
#eg = check_df.loc[(check_df['[EXPIRE_DATE]'] == '2022-02-28') & (check_df['[STRIKE]'] == 3900) & (check_df['[QUOTE_DATE]'] == '2022-12-29')]

eg = check_df.loc[check_df['[EXPIRE_DATE]'] == '2022-02-28']
#check_df.loc[check_df['[EXPIRE_DATE]'] == '2022-02-28'][500:510]

#_ = check_df.loc[(check_df['[EXPIRE_DATE]'] == '2022-02-28') & (check_df['[STRIKE]'] == 3900) & (check_df['[QUOTE_DATE]'] == '2022-12-30')]

In [ ]:
eg[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']][500:501]

In [ ]:
calc_daily_var[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']]

In [ ]:
eg[['[QUOTE_DATE]', '[EXPIRE_DATE]', '[UNDERLYING_LAST]', '[DTE]', '[C_DELTA]', '[C_LAST]', '[C_BID]', '[C_ASK]', '[STRIKE_DISTANCE]', '[STRIKE]', '[C_IV]']][1501:1502]

In [ ]:
eg['[STRIKE]'].iloc[1501]

In [ ]:
from bjerksund_stensland import bjerksund_stensland_call as bsc
from bjerksund_stensland import implied_volatility_call as ivc

In [ ]:
bsc(eg['[UNDERLYING_LAST]'].iloc[1501], eg['[STRIKE]'].iloc[1501], eg['[DTE]'].iloc[1501]/365, risk_free_rate=0.0045, volatility=0.17604)
## RISK FREE RATE 0.004-0.005 gets close to MID-PRICE of the option

In [ ]:
ivc(eg['[UNDERLYING_LAST]'].iloc[2501], eg['[STRIKE]'].iloc[2501], eg['[DTE]'].iloc[2501]/365, risk_free_rate=0.0045, option_price=(eg['[C_BID]'].iloc[2501]+eg['[C_ASK]'].iloc[2501])/2)

In [ ]:
synthe_X_df['[DTE]'] = synthe_X_df['[DTE]'].round()
synthe_X_df['[STRIKE]'] = synthe_X_df['[STRIKE]'].round()
synthe_X_df[-10:-5]

In [ ]:
synthe_X_df['[CALC_IV]'] = synthe_X_df.apply(lambda x: ivc(x['[UNDERLYING_LAST]'], x['[STRIKE]'], x['[DTE]']/365, 0.0045, (x['[C_BID]']+x['[C_ASK]'])/2), axis=1)

In [ ]:
synthe_X_df[-10:-3]

sp7_400[' [C_LAST]'].replace({' ':np.nan},inplace=True)
sp7_400[' [C_BID]'].replace({' ':np.nan},inplace=True)
sp7_400[' [C_ASK]'].replace({' ':np.nan},inplace=True)

sp7_400 = sp7_400.fillna(0)

sp7_400[' [C_LAST]'] = sp7_400[' [C_LAST]'].astype(np.float64)
sp7_400[' [C_BID]'] = sp7_400[' [C_BID]'].astype(np.float64)
sp7_400[' [C_ASK]'] = sp7_400[' [C_ASK]'].astype(np.float64)

# Data Prepare SPY Stock (2017-2019)

In [8]:
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from preprocess_TCN.gaussianize import *

In [9]:
spy_data = pd.read_csv('./data_options/SPY.csv')
spy_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,225.039993,225.830002,223.880005,225.240005,202.085266,91366500
1,2017-01-04,225.619995,226.750000,225.610001,226.580002,203.287506,78744400
2,2017-01-05,226.270004,226.580002,225.479996,226.399994,203.125992,78379000
3,2017-01-06,226.529999,227.750000,225.899994,227.210007,203.852722,71559900
4,2017-01-09,226.910004,227.070007,226.419998,226.460007,203.179855,46939700


### For 1 feature 

In [10]:
df = spy_data['Adj Close']

returns = df.shift(1)/df - 1
log_returns = np.log(df/df.shift(1))[1:].to_numpy().reshape(-1, 1)
standardScaler1 = StandardScaler()
standardScaler2 = StandardScaler()
gaussianize = Gaussianize()
log_returns_preprocessed = standardScaler2.fit_transform(gaussianize.fit_transform(standardScaler1.fit_transform(log_returns)))
data_size = log_returns.shape[0]
print(log_returns_preprocessed.shape)
print(data_size)

(752, 1)
752


In [13]:
# Save the Numpy arrays for training.
import random
random_ind, random_ind2 = random.randrange(0,log_returns_preprocessed.shape[0]-128), random.randrange(0,log_returns_preprocessed.shape[0]-128)
y_test = log_returns_preprocessed[random_ind:random_ind+127]
y_val = log_returns_preprocessed[random_ind2:random_ind2+127]
print(random_ind, random_ind2, y_test.shape, y_val.shape)

with open('tab-ddpm/tab-ddpm-main/data/stock/y_train.npy', 'wb') as f:
    np.save(f, log_returns_preprocessed)
with open('tab-ddpm/tab-ddpm-main/data/stock/X/X_num_train.npy', 'wb') as f:
    np.save(f, log_returns_preprocessed)
with open('tab-ddpm/tab-ddpm-main/data/stock/y_test.npy', 'wb') as f:
    np.save(f, y_test)
with open('tab-ddpm/tab-ddpm-main/data/stock/y_val.npy', 'wb') as f:
    np.save(f, y_val)


103 307 (127, 1) (127, 1)


### For n features

In [10]:
spy_data = spy_data[['Open', 'High', 'Low', 'Adj Close', 'Volume']]
log_returns = np.log(spy_data['Adj Close']/spy_data['Adj Close'].shift(1))[1:].to_numpy().reshape(-1, 1)
spy_data.shape

(753, 5)

In [11]:
spy_data = spy_data[1:]
spy_data['log_returns'] = log_returns
spy_data.shape

(752, 6)

In [12]:
# Save the Numpy arrays for training.
import random
random_ind, random_ind2 = random.randrange(0,spy_data.shape[0]-128), random.randrange(0,spy_data.shape[0]-128)
X_num_val = spy_data[random_ind:random_ind+127]
X_num_test = spy_data[random_ind2:random_ind2+127]
print(random_ind, random_ind2, X_num_val.shape, X_num_test.shape)

31 30 (127, 6) (127, 6)


In [13]:
y_train = spy_data.iloc[:,-1]
y_val = X_num_val.iloc[:,-1]
y_test = X_num_val.iloc[:,-1]

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(752,)
(127,)
(127,)


In [14]:
X_num_train = spy_data.iloc[:,:-1]
X_num_val = X_num_val.iloc[:,:-1]
X_num_test = X_num_test.iloc[:,:-1]

print(X_num_train.shape)
print(X_num_val.shape)
print(X_num_test.shape)

(752, 5)
(127, 5)
(127, 5)


In [20]:
y_test.iloc[:5]

32    0.001575
33    0.005937
34   -0.000888
35    0.000677
36    0.001268
Name: log_returns, dtype: float64

In [15]:
# Save the Numpy arrays for training.
with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/X_num_train.npy', 'wb') as f:
    np.save(f, X_num_train)
with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/X_num_test.npy', 'wb') as f:
    np.save(f, X_num_test)
with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/X_num_val.npy', 'wb') as f:
    np.save(f, X_num_val)

with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/y_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/y_test.npy', 'wb') as f:
    np.save(f, y_test)
with open('tab-ddpm/tab-ddpm-main/data/stock-nfeatures/y_val.npy', 'wb') as f:
    np.save(f, y_val)

### Eval for Stock with 1 feature

In [30]:
#### GET SYNTHETIC DATA
synthe_X_num = np.load('tab-ddpm/tab-ddpm-main/exp/stock/ddpm_mlp_best/X_num_train.npy')

synthe_y = np.load('tab-ddpm/tab-ddpm-main/exp/stock/ddpm_mlp_best/y_train.npy')

In [31]:
synthe_X_num

array([], shape=(127, 0), dtype=float64)

In [37]:
synthe_y[:5]

array([-1.55011986, -0.19795486,  1.08052938,  1.51370188, -2.48253531])

In [34]:
log_returns_preprocessed.shape

(752, 1)

In [39]:
y_val[:5]

array([[ 2.42351292],
       [-2.05496214],
       [-0.69604107],
       [ 1.65383531],
       [-2.28071775]])

In [41]:
# Trasform y_val
y_val_transformed = (y_val - y_val.mean(axis=0))/y_val.std(axis=0)
y_val_transformed = standardScaler2.inverse_transform(y_val_transformed)
y_val_transformed = np.array([gaussianize.inverse_transform(np.expand_dims(x, 1)) for x in y_val_transformed]).squeeze()
y_val_transformed = standardScaler1.inverse_transform(y_val_transformed.reshape(-1,1))
print(f'checking y: f{y_val_transformed.shape}')

checking y: f(127, 1)


In [42]:
y_val_transformed[:5]

array([[ 0.02735297],
       [-0.01902699],
       [-0.00335118],
       [ 0.01262557],
       [-0.02434983]])

In [45]:
# Trasform synthe_y
synthe_y_transformed = (synthe_y - synthe_y.mean(axis=0))/synthe_y.std(axis=0)
synthe_y_transformed = standardScaler2.inverse_transform(synthe_y_transformed.reshape(-1,1))
synthe_y_transformed = np.array([gaussianize.inverse_transform(np.expand_dims(x, 1)) for x in synthe_y_transformed]).squeeze()
synthe_y_transformed = standardScaler1.inverse_transform(synthe_y_transformed.reshape(-1,1))
print(f'checking y: f{synthe_y_transformed.shape}')

checking y: f(127, 1)


In [58]:
synthe_y_transformed.shape
y_val_transformed.shape
#synthe_y.shape
#y_val.shape

(127, 1)

In [47]:
from scipy.stats import wasserstein_distance

In [50]:
wasserstein_distance(y_val_transformed.reshape(-1), synthe_y_transformed.reshape(-1))

0.0007435944112121214

In [53]:
wasserstein_distance(y_val.reshape(-1), synthe_y)

0.10449159437589034

In [59]:
from preprocess.acf import *

windows = [1, 5, 20, 100]

#print(best_gen_path.shape)

for i in range(len(windows)):
    real_dist = rolling_window(y_val_transformed.reshape(-1), windows[i], sparse = not (windows[i] == 1)).sum(axis=0).ravel()
    fake_dist = rolling_window(synthe_y_transformed.reshape(-1), windows[i], sparse = not (windows[i] == 1)).sum(axis=0).ravel()
    #other_real_dist = rolling_window(test_against_itself.reshape(-1), windows[i], sparse = not (windows[i] == 1)).mean(axis=0).ravel()


    print(f'For window {windows[i]}, EMD_fake = {wasserstein_distance(real_dist, fake_dist)}')
    #print(f'For window {windows[i]}, EMD_otherReal = {wasserstein_distance(real_dist, other_real_dist)}')
    #print(fake_dist[:10], real_dist[:10])
    #print('YOOOO', fake_dist.shape, real_dist.shape)

    #print(np.array([fake_dist, real_dist], dtype='object')[0][185:195])
    #print(fake_dist[185:195], y[0][185:], y[1][:5], )
    print('------x-------x----------')

For window 1, EMD_fake = 0.0007435944112121214
------x-------x----------
For window 5, EMD_fake = 0.003358784637320827
------x-------x----------
For window 20, EMD_fake = 0.020055149423110125
------x-------x----------
For window 100, EMD_fake = 0.026364013672021694
------x-------x----------
